In [ ]:
!pip install openai==0.28.0

# Your original code with imports, functions, and main call

# Rest of the original code goes here...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 15.4 MB/s eta 0:00:00


In [ ]:
!pip install  tqdm tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.9 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


import os
directory_path = '/content/drive/Mathusha/MyDrive/Repair/src'
print(os.listdir(directory_path))  # Should list gen_solution_prompt.py among other files


In [5]:
import os
import sys
import importlib.util

# Check current working directory
print("Current Working Directory:", os.getcwd())

# Append the correct directory path
sys.path.append('/content/drive/MyDrive/Repair/src')

# Verify the existence of the file
file_path = '/content/drive/MyDrive/Repair/src/gen_solution_prompt.py'
print("File Exists:", os.path.exists(file_path))

# Check if the file is in the path
for p in sys.path:
    print(f"Checking path: {p}")
    if os.path.exists(os.path.join(p, 'gen_solution_prompt.py')):
        print(f"Found 'gen_solution_prompt.py' in path: {p}")

# Try importing the module using importlib
module_name = 'gen_solution_prompt'
module_path = '/content/drive/MyDrive/Repair/src/gen_solution_prompt.py'

try:
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    gen_solution_prompt = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = gen_solution_prompt
    spec.loader.exec_module(gen_solution_prompt)
    print(f"Module '{module_name}' imported successfully!")
    sf_construct_prompt = gen_solution_prompt.sf_construct_prompt
    print("Function 'sf_construct_prompt' imported successfully!")
except Exception as e:
    print(f"Error importing module using importlib: {e}")


Current Working Directory: /content
File Exists: False
Checking path: /content
Checking path: /env/python
Checking path: /usr/lib/python310.zip
Checking path: /usr/lib/python3.10
Checking path: /usr/lib/python3.10/lib-dynload
Checking path: 
Checking path: /usr/local/lib/python3.10/dist-packages
Checking path: /usr/lib/python3/dist-packages
Checking path: /usr/local/lib/python3.10/dist-packages/IPython/extensions
Checking path: /usr/local/lib/python3.10/dist-packages/setuptools/_vendor
Checking path: /root/.ipython
Checking path: /content/drive/MyDrive/Repair/src
Error importing module using importlib: [Errno 2] No such file or directory: '/content/drive/MyDrive/Repair/src/gen_solution_prompt.py'


In [ ]:
import time
import re
import os
import json
import openai
from tqdm import tqdm

# Import the module using importlib if the standard import fails
import sys
sys.path.append('/content/drive/MyDrive/Repair/src')
import importlib.util

module_name = 'gen_solution_prompt'
module_path = '/content/drive/MyDrive/Repair/src/gen_solution_prompt.py'

try:
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    gen_solution_prompt = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = gen_solution_prompt
    spec.loader.exec_module(gen_solution_prompt)
    sf_construct_prompt = gen_solution_prompt.sf_construct_prompt
    print("Module and function imported successfully!")
except Exception as e:
    print(f"Error importing module using importlib: {e}")

# Set your OpenAI API key
openai.api_key = 'sk-proj-rynubDWcMiHlu7h2mRvsT3BlbkFJTPEpaQQ3GCdo2YVe4LGz'

class SolInfo:
    def __init__(self, dataset_path, solution_path, extracted_solution_path, sample_size, target_bug):
        self.dataset = self.load_dataset(dataset_path, target_bug)
        self.solution_path = solution_path
        self.extracted_solution_path = extracted_solution_path or self.generate_extracted_path(solution_path)
        self.sample_size = sample_size
        self.target_bug = target_bug

    @staticmethod
    def load_dataset(dataset_path, target_bug):
        try:
            with open(dataset_path, 'r') as f:
                dataset = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"The file {dataset_path} was not found.")
        except json.JSONDecodeError:
            raise ValueError(f"The file {dataset_path} is not a valid JSON file.")

        if target_bug is not None:
            if target_bug in dataset:
                return {target_bug: dataset[target_bug]}
            else:
                raise KeyError(f"Target bug '{target_bug}' not found in the dataset.")
        return dataset

    @staticmethod
    def generate_extracted_path(solution_path):
        base, ext = os.path.splitext(solution_path)
        return f"{base}_extracted{ext}"

def query_model(prompt, sample_size):
    delay = 10
    while True:
        try:
            response = api_gpt3_5_response(prompt, sample_size)
            break
        except openai.error.OpenAIError as e:
            if "Please reduce " in str(e):
                raise ValueError("Over Length")
            print(f"OpenAI API returned an API Error: {e}")
            time.sleep(delay)
        except Exception as e:
            print(f"Exception in api_query_model {e}")
            if "Please reduce " in str(e):
                raise ValueError("Over Length")
            time.sleep(delay)

    return [choice['message']['content'] for choice in response.choices]

def api_gpt3_5_response(prompt, n):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        n=n,
        temperature=0.8
    )
    return response

def split_solutions(text):
    pattern = r'Suggestion \d+:.*?(?=Suggestion \d+:|$)'
    solutions = re.findall(pattern, text, re.DOTALL)

    return [solution.split(':', 1)[1].strip() if ':' in solution else solution.strip() for solution in solutions]

def extract_root_cause(text):
    pattern = r'Root Cause:.*?(?=Suggestion \d+:|$)'
    match = re.search(pattern, text, re.DOTALL)

    return match.group().split(':', 1)[1].strip() if match else None

def get_solutions(sol_info):
    solutions = {}
    for bug_name in tqdm(sol_info.dataset.keys()):
        prompt = sf_construct_prompt(sol_info.dataset, bug_name)
        solutions[bug_name] = {
            'prompt': prompt,
            'solutions': query_model(prompt, sol_info.sample_size)
        }
    return solutions

def extract_solutions(raw_solutions):
    extracted_solutions = {}
    for bug_name, data in raw_solutions.items():
        extracted_solutions[bug_name] = {}
        for solution in data['solutions']:
            split_solution_lst = split_solutions(solution)
            curr_root_cause = extract_root_cause(solution)
            if curr_root_cause not in extracted_solutions[bug_name]:
                extracted_solutions[bug_name][curr_root_cause] = []
            extracted_solutions[bug_name][curr_root_cause].extend(split_solution_lst)
    return extracted_solutions

def main(dataset_path, solution_path, extracted_solution_path, sample_size, target_bug):
    sol_info = SolInfo(dataset_path, solution_path, extracted_solution_path, sample_size, target_bug)
    solutions = get_solutions(sol_info)
    with open(sol_info.solution_path, 'w') as f:
        json.dump(solutions, f, indent=2)
    extracted_solutions = extract_solutions(solutions)
    with open(sol_info.extracted_solution_path, 'w') as f:
        json.dump(extracted_solutions, f, indent=2)

# Define paths and parameters
dataset_path = '/content/drive/MyDrive/Repair/defects4j-sf.json'
solution_path = '/content/drive/MyDrive/Repair/math27_solution.json'
extracted_solution_path = '/content/drive/MyDrive/Repair/math27_solution_extracted.json'
sample_size = 1
target_bug = 'Math-27'

# Call the main function with defined parameters
main(dataset_path, solution_path, extracted_solution_path, sample_size, target_bug)



Module and function imported successfully!


100%|██████████| 1/1 [00:06<00:00,  6.27s/it]


In [ ]:
import sys
import os

# Add the directory containing the module to the Python path
module_path = '/content/drive/MyDrive/Repair/src'
if module_path not in sys.path:
    sys.path.append(module_path)

print("Current sys.path:")
print(sys.path)

# List contents of the module directory to verify the presence of gen_patch_prompt.py
print("Contents of the module directory:")
print(os.listdir(module_path))

# Attempt to import the module
try:
    from gen_patch_prompt import sf_build_apr_prompt_auto
    print("Module imported successfully.")
except ModuleNotFoundError as e:
    print(f"Error: {e}")

import re
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_format_prompt = '''Return your fixed function surrounded with ```java\\n```.\n@@ Instruction\n{apr_prompt}\n@@ Response'''

def extract_test_method(testcase_lst):
    method_info_lst = []
    bracket_cnt = 0
    bracket_flag = False
    for line in testcase_lst:
        method_info_lst.append(line)
        bracket_cnt += line.count('{')
        if bracket_cnt:
            bracket_flag = True
        bracket_cnt -= line.count('}')
        if bracket_cnt == 0 and bracket_flag:
            return method_info_lst
    return None

def extract_all_patch_codes(orig_patch, dataset, bug_name):
    patch_code_lst = []
    code_patch_pattern = r'```(?:java\n)?(.*?)\n```'
    extracted_lst = re.findall(code_patch_pattern, orig_patch, re.DOTALL)
    function_name = ' ' + dataset[bug_name]['method_signature']['method_name'] + '('
    function_return_type = dataset[bug_name]['method_signature']['return_type']

    if extracted_lst:
        for patch_code in extracted_lst:
            if function_name in patch_code and function_return_type in patch_code:
                patch_code_lst.append(patch_code)
        if len(patch_code_lst) > 0:
            return patch_code_lst

    orig_patch_lines = orig_patch.split('\n')
    for idx in range(len(orig_patch_lines) - 1, -1, -1):
        curr_rline = orig_patch_lines[idx]
        if function_name not in curr_rline or function_return_type not in curr_rline:
            continue
        patch_code = extract_test_method(orig_patch_lines[idx:])
        if patch_code:
            patch_code_lst.append('\n'.join(patch_code))
            break
    return patch_code_lst

def extract_patch(dataset, raw_patch_result):
    extracted_patch_result = {}
    for bug_name, data in raw_patch_result.items():
        extracted_patch_result[bug_name] = {'prompt': data['prompt'], 'patches': []}
        for raw_patch in data['patches']:
            extracted_patches = extract_all_patch_codes(raw_patch, dataset, bug_name)
            for extracted_patch in extracted_patches:
                if extracted_patch.startswith(':'):
                    extracted_patch = extracted_patch[1:]
                elif extracted_patch.startswith('@@ Response:'):
                    extracted_patch = extracted_patch[12:]
                extracted_patch_result[bug_name]['patches'].append(extracted_patch)
    return extracted_patch_result

class AprInfo:
    def __init__(self, dataset_path, suggestions_path, out_path, target_bug):
        self.dataset = self.load_json(dataset_path)
        self.suggestions = self.load_json(suggestions_path)
        if target_bug is not None:
            self.dataset = {target_bug: self.dataset[target_bug]}
            self.suggestions = {target_bug: self.suggestions[target_bug]}
        self.out_path = out_path
        self.target_bug = target_bug

    @staticmethod
    def load_json(path):
        with open(path, 'r') as f:
            return json.load(f)

def opensrc_model_apr(apr_info):
    try:
        tokenizer = AutoTokenizer.from_pretrained("ise-uiuc/Magicoder-S-CL-7B")
        model = AutoModelForCausalLM.from_pretrained("ise-uiuc/Magicoder-S-CL-7B")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
    except Exception as e:
        print(f'Error loading model or tokenizer: {e}')
        return

    patches = {}
    for bug_name, bug_data in apr_info.dataset.items():
        buggy_token_len = len(tokenizer.encode(bug_data['buggy']))
        curr_max_new_token = max(256, 2 * buggy_token_len)

        curr_patch = {'prompt': [], 'patches': []}

        for root_cause, suggestions in apr_info.suggestions[bug_name].items():
            for suggestion in suggestions:
                apr_prompt = sf_build_apr_prompt_auto(bug_data['buggy'], root_cause, suggestion)
                prompt = model_format_prompt.format(apr_prompt=apr_prompt.strip())
                prompt_token_len = len(tokenizer.encode(prompt))
                if prompt_token_len > 4000:
                    continue

                curr_patch['prompt'].append(apr_prompt)
                suggest_patch = []
                batch_size = 1
                if tokenizer.pad_token is None:
                    tokenizer.pad_token = tokenizer.eos_token
                pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
                inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

                while len(suggest_patch) < 5:
                    try:
                        outputs = model.generate(
                            inputs["input_ids"],
                            max_new_tokens=curr_max_new_token,
                            attention_mask=inputs["attention_mask"],
                            pad_token_id=pad_token_id,
                            do_sample=True,
                            top_p=0.9,
                            temperature=0.8,
                            num_return_sequences=batch_size,
                        )
                        for output in outputs:
                            generated_text = tokenizer.decode(output, skip_special_tokens=True)
                            prompt_end_idx = generated_text.find(apr_prompt.strip()) + len(apr_prompt.strip())
                            fixed_result = generated_text[prompt_end_idx:].trip() if prompt_end_idx != -1 else generated_text
                            fixed_result = fixed_result.replace('[/INST]  ', '')
                            suggest_patch.append(fixed_result)

                    except RuntimeError as e:
                        if 'out of memory' in str(e):
                            batch_size = batch_size // 2
                            if batch_size == 0:
                                break
                        else:
                            raise e

                curr_patch['patches'].extend(suggest_patch)
        patches[bug_name] = curr_patch

    return patches

def main():
    dataset_path = '/content/drive/MyDrive/Repair/defects4j-sf.json'
    suggestions_path = '/content/drive/MyDrive/Repair/math27_solution_extracted.json'
    out_path = '/content/drive/MyDrive/Repair/math27_patches.json'
    target_bug = 'Math-27'

    apr_info = AprInfo(dataset_path, suggestions_path, out_path, target_bug)
    apr_result = opensrc_model_apr(apr_info)
    if apr_result:
        apr_result = extract_patch(apr_info.dataset, apr_result)
        with open(apr_info.out_path, 'w') as f:
            json.dump(apr_result, f, indent=2)

if __name__ == "__main__":
    main()


Current sys.path:
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/drive/MyDrive/Repair/src', '/content/drive/MyDrive/Repair/src']
Contents of the module directory:
['mf_val_d4j.py', 'gen_patch_prompt.py', 'sf_gen_solution.py', 'mf_gen_patch.py', 'dataset_preprocess.py', 'mf_gen_solution.py', 'sf_val_d4j.py', '__pycache__', 'gen_solution_prompt.py', 'sf_gen_patch.py', 'Suggestion.ipynb']
Module imported successfully.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00003-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
import gc
gc.collect()
